## Santander Customer Satisfaction
### Source
https://www.kaggle.com/competitions/santander-customer-satisfaction/overview

### Tasks
Predict customer satisfaction

### Evaluation
ROC-AUC Curve (small portion of dissatisfaction)

In [11]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib as plt
import seaborn as sns
import warnings

### 데이터 전처리

In [12]:
base_path='/Users/minha/ai/ai-practice/python-machine-learning-guide/classification/santander-customer-satisfaction/'

warnings.filterwarnings('ignore')
customer_df = pd.read_csv(f'{base_path}train.csv', encoding='latin-1')
print('data shape:', customer_df.shape)
customer_df.head(3) # 371 features

data shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [13]:
customer_df.info() # feature data type is all number: 111 of float and 260 of int, no NULL

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [14]:
## 불만족 고객의 비율 계산
# value_count(): 해당 컬럼의 값 빈도수 (0과 1의 빈도수)
# count(): 총 개수

print(customer_df['TARGET'].value_counts())

unsatisfied_customer =  customer_df[customer_df['TARGET'] == 1].TARGET.count()
total_customer = customer_df.TARGET.count()

print(f'unsatisfied customers ratio: {unsatisfied_customer/total_customer}')

TARGET
0    73012
1     3008
Name: count, dtype: int64
unsatisfied customers ratio: 0.0395685345961589


In [15]:
## 각 feature 값의 분포
# var3의 min 값이 -999999으로 NaN이거나 특정 예외 값을 변환
customer_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [16]:
## 결측값 확인
# var3 컬럼의 상위 10개 인덱스의 값 빈도수
# -999999가 116개로 NaN값이나 특정 예외 값을 해당 숫자로 변환
print(customer_df.var3.value_counts()[:10]) 

var3
 2         74165
 8           138
-999999      116
 9           110
 3           108
 1           105
 13           98
 7            97
 4            86
 12           85
Name: count, dtype: int64


In [17]:
# var3는 숫자형이기 때문에 그 편차가 심해 가장 값이 많은 2로 변환 (최빈값으로 결측값 보완)
customer_df['var3'].replace(-999999, 2, inplace=True)

# 사용하지 않는 ID 컬럼 제거
customer_df.drop('ID', axis=1, inplace=True)

In [19]:
## feature와 label 분리
# iloc[row, column] -> [:, :-1] 모든 row와, 마지막 컬럼을 제외한 모든 column
features = customer_df.iloc[:, :-1] 
labels = customer_df.iloc[:, -1]

print(f'feature data shape {features.shape}')

feature data shape (76020, 369)


In [22]:
## 학습 세트와 테스트 세트로 분리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0) 

In [32]:
## 학습, 테스트 세트 레이블 분포 확인
# 불만족 값이 각각 4%로 데이터 분포 확인
train_count = y_train.count()
test_count = y_test.count()

print(f'학습 세트: {x_train.shape}, 테스트 세트: {x_test.shape}')
print(f'학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_count)
print(f'테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_count)

학습 세트: (60816, 369), 테스트 세트: (15204, 369)
학습 세트 레이블 값 분포 비율
TARGET
0    0.960964
1    0.039036
Name: count, dtype: float64
테스트 세트 레이블 값 분포 비율
TARGET
0    0.9583
1    0.0417
Name: count, dtype: float64


In [34]:
## XGBoost의 조기 중단의 검증 세트
# train set을 다시 학습과 검증 세트로 분리
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=0) 

### XGBoost 모델 학습

In [44]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

## XGBoost 모델 (성능 0.84)
# n_estimators: The number of boosting rounds. it will train up to 500 trees.
# randome_state: reproducibility by setting the seed for the random number generator. (재현성을 위해 난수 시드를 설정)\
# eval_metric: 평가지표로 AUC 설정
xgb_classifier = XGBClassifier(n_estimators=500, learning_rate=0.05, random_state=156, early_stopping_rounds=100, eval_metric="auc")

xgb_classifier.fit(x_tr, y_tr, eval_set=[(x_tr, y_tr), (x_val, y_val)])

soc_score = roc_auc_score(y_test, xgb_classifier.predict_proba(x_test)[:, 1])
print(f'ROC AUC: {soc_score}')

[0]	validation_0-auc:0.83603	validation_1-auc:0.80978
[1]	validation_0-auc:0.83783	validation_1-auc:0.81126
[2]	validation_0-auc:0.83892	validation_1-auc:0.81192
[3]	validation_0-auc:0.84090	validation_1-auc:0.81349
[4]	validation_0-auc:0.84187	validation_1-auc:0.81356
[5]	validation_0-auc:0.84355	validation_1-auc:0.81422
[6]	validation_0-auc:0.84534	validation_1-auc:0.81550
[7]	validation_0-auc:0.84658	validation_1-auc:0.81623
[8]	validation_0-auc:0.84727	validation_1-auc:0.81669
[9]	validation_0-auc:0.84794	validation_1-auc:0.81668
[10]	validation_0-auc:0.84980	validation_1-auc:0.81828
[11]	validation_0-auc:0.85105	validation_1-auc:0.81965
[12]	validation_0-auc:0.85217	validation_1-auc:0.81999
[13]	validation_0-auc:0.85301	validation_1-auc:0.82049
[14]	validation_0-auc:0.85558	validation_1-auc:0.82210
[15]	validation_0-auc:0.85662	validation_1-auc:0.82288
[16]	validation_0-auc:0.85811	validation_1-auc:0.82380
[17]	validation_0-auc:0.85864	validation_1-auc:0.82365
[18]	validation_0-au